# Baseline Gender Biased Token Classifiers

### Target: Label Categories

### Word Embeddings: GloVe

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/model_input/`
    * Prediction Data: under directory `../data/token_clf_data/model_output`
* Multilabel classification
    * 3 categories of labels: Linguistic, Person Name, Contextual
* Word Embeddings
    * GloVe (trained on English Wikipedia dump)

***

### Table of Contents

**[0.](#0) Preprocessing**

**[1.](#1) Baseline Model**

**[2.](#2) Hyperparameter Optimization**

**[3.](#3) Error Analysis**

***

Load necessary libraries:

In [1]:
# For custom functions and variables
import utils, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For visualization
import matplotlib.pyplot as plt

# For preprocessing
from nltk.stem import WordNetLemmatizer
import scipy.stats
from gensim.models import FastText
from gensim import utils as gensim_utils
from gensim.test.utils import get_tmpfile

# For classification
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# For evaluation
from collections import Counter
from sklearn.metrics import classification_report, make_scorer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay#, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, f1_score
from intervaltree import Interval, IntervalTree

<a id="0"></a>
## 0. Preprocessing

Load the train and validation (dev) data:

In [2]:
df_train = pd.read_csv(config.tokc_path+"model_input/token_train.csv", index_col=0)
df_dev = pd.read_csv(config.tokc_path+"model_input/token_validate.csv", index_col=0)
print(df_train.shape, df_dev.shape)
df_train.head()

(467564, 10) (157740, 10)


,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,subset
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,train
4,1,1,99999,4,:,"(22, 23)",:,O,Title,train
5,1,1,99999,5,Papers,"(24, 30)",NNS,O,Title,train
6,1,1,99999,6,of,"(31, 33)",IN,O,Title,train
7,1,1,14384,7,The,"(34, 37)",DT,B-Unknown,Title,train


Drop duplicate rows with all but the same annotation ID:

In [3]:
df_train = df_train.drop(columns=["ann_id"])
df_train = df_train.drop_duplicates()
df_dev = df_dev.drop(columns=["ann_id"])
df_dev = df_dev.drop_duplicates()
print(df_train.shape, df_dev.shape)

(463441, 9) (156146, 9)


Remove Non-binary labels as these were mistaken labels identified early on that were meant to be excluded, and because only one token has this label, it prevents the data from being input into the models with cross-validation.

In [4]:
df_train = df_train.loc[df_train.tag != "B-Nonbinary"]
df_train = df_train.loc[df_train.tag != "I-Nonbinary"]

In [5]:
df_train.shape

(463439, 9)

***

#### Label Categories

Add the annotation label categories as a column of higher-level Inside-Outside-Beginning (IOB) tags so they can be used as targets:

In [6]:
df_train = utils.addCategoryTagColumn(df_train)
# df_train.head(20)

In [7]:
df_dev = utils.addCategoryTagColumn(df_dev)
# df_dev.head()

Remove columns that won't be used as features for the classifiers and remove any duplicate rows that remain:

In [8]:
cols_to_keep = ["sentence_id", "token_id", "pos", "token", "tag_cat"]

In [9]:
df_train = df_train[cols_to_keep]
df_train = df_train.drop_duplicates()
df_dev = df_dev[cols_to_keep]
df_dev = df_dev.drop_duplicates()
# df_train.head(20)

Create columns for each category so they can be used as three separate targets:

In [25]:
ling_cat_tags = ["B-Linguistic", "I-Linguistic"]
df_train_ling = df_train.loc[df_train.tag_cat.isin(ling_cat_tags)]
df_dev_ling = df_dev.loc[df_dev.tag_cat.isin(ling_cat_tags)]

In [26]:
pers_cat_tags = ["B-Person-Name", "I-Person-Name"]
df_train_pers = df_train.loc[df_train.tag_cat.isin(pers_cat_tags)]
df_dev_pers = df_dev.loc[df_dev.tag_cat.isin(pers_cat_tags)]

In [27]:
cont_cat_tags = ["B-Contextual", "I-Contextual"]
df_train_cont = df_train.loc[df_train.tag_cat.isin(cont_cat_tags)]
df_dev_cont = df_dev.loc[df_dev.tag_cat.isin(cont_cat_tags)]

In [28]:
df_train = (df_train.drop(columns=["tag_cat"])).drop_duplicates()
df_dev = (df_dev.drop(columns=["tag_cat"])).drop_duplicates()

In [29]:
join_cols = ["sentence_id", "token_id", "pos", "token"]

In [31]:
df_train = df_train.join(df_train_ling.set_index(join_cols), on=join_cols, how="outer")
df_train = df_train.join(df_train_pers.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_personname")
df_train = df_train.join(df_train_cont.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_contextual")
df_train = df_train.rename(columns={"tag_cat":"tag_cat_linguistic"})
# df_train.head(30)  # Should have one row per token!

In [32]:
df_dev = df_dev.join(df_dev_ling.set_index(join_cols), on=join_cols, how="outer")
df_dev = df_dev.join(df_dev_pers.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_personname")
df_dev = df_dev.join(df_dev_cont.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_contextual")
df_dev = df_dev.rename(columns={"tag_cat":"tag_cat_linguistic"})
# df_dev.head(30)

In [20]:
# df_train.tail(30)

,sentence_id,token_id,pos,token,tag_cat_linguistic,tag_cat_personname,tag_cat_contextual
779229,42027,753891,NN,treatment,NaN,NaN,NaN
779230,42027,753892,IN,of,NaN,NaN,NaN
779231,42027,753893,NN,homosexuality,NaN,NaN,NaN
779232,42027,753894,IN,in,NaN,NaN,NaN
779233,42027,753895,JJ,contemporary,NaN,NaN,NaN
779234,42027,753896,JJ,medical,NaN,NaN,NaN
779235,42027,753897,NNS,journals,NaN,NaN,NaN
779236,42027,753898,CC,and,NaN,NaN,NaN
779237,42027,753899,NNS,books,NaN,NaN,NaN
779238,42027,753900,",",",",NaN,NaN,NaN


**REMEMBER:** check that model input data created on correct subset of files - no stereotypes about homosexuality "offences" or "medical treatment" of homosexuality??? 

Replace the `tag_cat_` columns' `nan` values with `'O'`:

In [33]:
tag_cat_cols = ["tag_cat_linguistic", "tag_cat_personname", "tag_cat_contextual"]
df_train[tag_cat_cols] = df_train[tag_cat_cols].fillna('O')
df_dev[tag_cat_cols] = df_dev[tag_cat_cols].fillna('O')
df_dev.head()

,sentence_id,token_id,pos,token,tag_cat_linguistic,tag_cat_personname,tag_cat_contextual
172,5,154,IN,After,O,O,O
173,5,155,PRP$,his,B-Linguistic,O,O
174,5,156,NN,ordination,O,O,O
175,5,157,PRP,he,B-Linguistic,O,O
176,5,158,VBD,spent,O,O,O


Group the data by sentence, so the token column becomes a list of tokens for each sentence:

In [35]:
df_train_grouped = utils.implodeDataFrame(df_train, ["sentence_id"])
df_dev_grouped = utils.implodeDataFrame(df_dev, ["sentence_id"])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped.head()

,token_id,pos,sentence,tag_cat_linguistic,tag_cat_personname,tag_cat_contextual
sentence_id,,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, B-Contextual, I-Co..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[O, O, O]","[O, O, O]","[O, O, O]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, O, B-Pers...","[O, O, B-Contextual, I-Contextual, I-Contextua..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, I-Person-...","[O, O, O, O, O, O, O, O, O, O, B-Contextual, O..."


Zip the POS and category tags together with the tokens so each sentence item is a tuple: `(TOKEN, POS-TAG, CATEGORY-TAG)`

In [41]:
train_sentences_ling = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_linguistic")
print(train_sentences_ling[0][:3])
dev_sentences_ling = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_linguistic")
print(dev_sentences_ling[0][:3])
train_sentences_pers = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_personname")
dev_sentences_pers = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_personname")
train_sentences_cont = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_contextual")
dev_sentences_cont = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_contextual")

[('Title', 'NN', 'O'), (':', ':', 'O'), ('Papers', 'NNS', 'O')]
[('After', 'IN', 'O'), ('his', 'PRP$', 'B-Linguistic'), ('ordination', 'NN', 'O')]


#### Word Embeddings

Use the custom fastText word embeddings, trained on the entire dataset of descriptive metadata from the Archives (harvested in October 2020) using the Continuous Bag-of-Words (CBOW) algorithm.  Subword embeddings (for subwords from 2 to 6 characters long, inclusive) are used to infer the embeddings for out-of-vocabulary (OOV) words.

Use the word embedding model trained on lowercased text to 100 dimensions: 

In [42]:
file_name = get_tmpfile(config.tokc_path+"fasttext100_lowercased.model")
embedding_model = FastText.load(file_name)

In [43]:
vocabulary = list(df_train.token.unique())
vocabulary_lowercased = [token.lower() for token in vocabulary]
vocabulary_lowercased = list(set(vocabulary_lowercased))
print("Vocabulary size:", len(vocabulary))
print("Lowercased vocabulary size:", len(vocabulary_lowercased))

Vocabulary size: 35968
Lowercased vocabulary size: 31335


Define feature dictionaries:

In [16]:
# Get a vector representation of a token from a fastText word embedding model
def extractEmbedding(token, fasttext_model=embedding_model):
    if token.isalpha():
        token = token.lower()
    embedding = fasttext_model.wv[token]
    return embedding

def extractTokenFeatures(sentence, i):
    token = sentence[i][0]
    pos = sentence[i][1]
    features = {
        'bias': 1.0,
        'pos': pos,
        'pos[:2]': pos[:2],
        'token': token
    }
    
    # Add each value in a token's word embedding as a separate feature
    embedding = extractEmbedding(token)
    for i,n in enumerate(embedding):
        features['e{}'.format(i)] = n
    
    # Record whether a token is the first or last token of a sentence
    if i == 0:
        features['START'] = True
    elif i == (len(sentence) - 1):
        features['END'] = True
    
    return features

def extractSentenceFeatures(sentence):
    return [extractTokenFeatures(sentence, i) for i in range(len(sentence))]

def extractSentenceTargets(sentence):
    return [tag for token, pos, tag in sentence]

def extractSentenceTokens(sentence):
    return [token for token, pos, tag in sentence]

In [18]:
# extractSentenceFeatures(train_sentences[0])[5]

*References:*
* *https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html*
* *https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training*

<a id="1"></a>
## 1. Baseline Models

### Linguistic

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Linguistic label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Linguistic** category of tags:

In [44]:
train_sentences = train_sentences_ling
dev_sentences = dev_sentences_ling

In [45]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [46]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
# Available algorithms with sklearn_crfsuite are:
#     'lbfgs' - Gradient descent using the L-BFGS method
#     'l2sgd' - Stochastic Gradient Descent with L2 regularization term
#     'ap' - Averaged Perceptron
#     'pa' - Passive Aggressive (PA)
#     'arow' - Adaptive Regularization Of Weight Vector (AROW)

In [47]:
# clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100) #iterations unlimited
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[2], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[3], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[4], max_iterations=100)           # max iterations allowed

In [48]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

In [28]:
# sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100)- ## should be able to see transition probabilities (if 5 labels, matrix of 5)

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [49]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Linguistic', 'I-Linguistic']


#### Predict

In [50]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation

In [51]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.678230100682343
  - Prec: 0.7553309858774123
  - Rec 0.6287799791449427


### Person Name

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Person-Name label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Person Name** category of tags:

In [52]:
train_sentences = train_sentences_pers
dev_sentences = dev_sentences_pers

In [53]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [46]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']

In [54]:
# clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100) #iterations unlimited
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[2], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[3], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[4], max_iterations=100)           # max iterations allowed

In [55]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [56]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Person-Name', 'I-Person-Name']


#### Predict

In [57]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation

In [58]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.4946309099465871
  - Prec: 0.8145205069894943
  - Rec 0.35538592027141647


<a id="1"></a>
### Contextual

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Contextual label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Contextual** category of tags:

In [44]:
train_sentences = train_sentences_ling
dev_sentences = dev_sentences_ling

In [45]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [46]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
# Available algorithms with sklearn_crfsuite are:
#     'lbfgs' - Gradient descent using the L-BFGS method
#     'l2sgd' - Stochastic Gradient Descent with L2 regularization term
#     'ap' - Averaged Perceptron
#     'pa' - Passive Aggressive (PA)
#     'arow' - Adaptive Regularization Of Weight Vector (AROW)

In [47]:
# clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100) #iterations unlimited
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[2], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[3], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[4], max_iterations=100)           # max iterations allowed

In [48]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

In [28]:
# sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100)- ## should be able to see transition probabilities (if 5 labels, matrix of 5)

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [49]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Linguistic', 'I-Linguistic']


#### Predict

In [50]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation

In [51]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.678230100682343
  - Prec: 0.7553309858774123
  - Rec 0.6287799791449427


The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

As with the manual annotation evaluation, we want to evaluate the predictions more loosely, considering overlapping text spans in addition to exactly matching text spans.  Save the predictions for each token and then use IntervalTree to evaluate performance considering overlapping labels, rather than only exactly matching labels.

In [58]:
# dev_sentences[0]
df_dev_grouped = df_dev_grouped.rename(columns={"tag_cat":"tag_cat_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_cat_predicted", y_pred)
df_dev_grouped.head()

,token_id,pos,sentence,tag_cat_expected,tag_cat_predicted
sentence_id,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[O, O, O]","[O, O, O]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
18,"[498, 499, 500, 500, 501, 501, 502, 503, 503, ...","[IN, CD, NNP, NNP, NNP, NNP, VBD, NNP, NNP, NN...","[In, 1941, Tom, Tom, Allan, Allan, married, Ja...","[O, O, B-Person-Name, B-Contextual, I-Person-N...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[O, O, B-Person-Name, I-Person-Name, I-Person-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [67]:
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns))
df_dev_exploded = df_dev_exploded.rename(columns={"sentence":"token"})
df_dev_exploded.head()

,token_id,pos,token,tag_cat_expected,tag_cat_predicted
sentence_id,,,,,
5,154,IN,After,O,O
5,155,PRP$,his,B-Linguistic,B-Linguistic
5,156,NN,ordination,O,O
5,157,PRP,he,B-Linguistic,B-Linguistic
5,158,VBD,spent,O,O


In [65]:
print(df_dev_exploded.shape)
print(df_dev.shape)

(154935, 5)
(154935, 5)


In [68]:
df_dev_grouped.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-CustomFastText_grouped.csv")
df_dev_exploded.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-CustomFastText.csv")

##### Loose Evaluation

Using the model with one of the best performing algorithms, Stochastic Gradient Descent with L2 regularization (`l2sgd`),  conduct a loose evaluation of the model's performance (as the manual annotation were evaluated).

To try to improve the model's performance, use cross-validation and randomized search for choosing regularization parameters:

In [125]:
folds = 3  # 3-fold cross validation

In [126]:
clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2 = 0.1, max_iterations=100) #, all_possible_transitions=True)